In [1]:
!tar -xvzf /content/drive/MyDrive/detection.tar.gz

detection/test/images/dataset2_back_630.png
detection/train/masks/20150919_174151_image11.png
detection/train/images/20150919_174730_image161.png
detection/train/masks/20150921_131346_image1121.png
detection/train/masks/20150921_131234_image546.png
detection/test/images/dataset3_front_510.png
detection/train/images/20150921_131346_image1121.png
detection/train/images/
detection/train/masks/20150921_132245_image396.png
detection/train/masks/20150921_131453_image676.png
detection/train/masks/20150921_131833_image296.png
detection/train/images/20150919_174151_image301.png
detection/train/images/20150921_132038_image76.png
detection/train/masks/20150921_131453_image1196.png
detection/train/images/20150921_131346_image51.png
detection/train/masks/20150921_132038_image1241.png
detection/test/images/dataset4_front_900.png
detection/test/images/dataset4_front_1020.png
detection/train/images/20150919_174730_image216.png
detection/train/images/20150921_131453_image1196.png
detection/train/images

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 4.3 MB/s eta 0:00:00


In [3]:
!mkdir -p /content/data/images/train
!mkdir -p /content/data/labels/train


In [4]:
import os
import numpy as np
import torch
from PIL import Image
import re

class AppleDataset(object):
    def __init__(self, root_dir, save_dir, transforms):
        self.root_dir = root_dir
        self.save_dir = save_dir
        self.transforms = transforms

        # Load all image and mask files, sorting them to ensure they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root_dir, "masks"))))

    def get_item(self, idx):
        # Load images and masks
        img_path = os.path.join(self.root_dir, "images", self.imgs[idx])
        mask_path = os.path.join(self.root_dir, "masks", self.masks[idx])

        img_name_indl = img_path.rfind("/") + 1
        img_name_indr = img_path.rfind(".")

        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)     # Each color of mask corresponds to a different instance with 0 being the background

        # Convert the PIL image to np array
        mask = np.array(mask)
        obj_ids = np.unique(mask)

        # Remove background id
        obj_ids = obj_ids[1:]

        # Split the color-encoded masks into a set of binary masks
        masks = mask == obj_ids[:, None, None]

        # Get bbox coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        h, w = mask.shape

        for ii in range(num_objs):
            pos = np.where(masks[ii])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])

            if xmin == xmax or ymin == ymax:
                continue

            xmin = np.clip(xmin, a_min=0, a_max=w)
            xmax = np.clip(xmax, a_min=0, a_max=w)
            ymin = np.clip(ymin, a_min=0, a_max=h)
            ymax = np.clip(ymax, a_min=0, a_max=h)

            xc = float(((xmin + xmax)/2)/w)
            yc = float(((ymin + ymax)/2)/h)
            width =  float(((xmax - xmin))/w)
            height = float((ymax - ymin)/h)
#            print(xmin, xmax, ymin, ymax)
            boxes.append([xc, yc, width, height])

        with open(self.save_dir + img_path[img_name_indl:img_name_indr] + ".txt", "w") as f:
          for box in boxes:
              xc, yc, w, h = box[0], box[1], box[2], box[3]
              coord = "0 " + str(xc) + " " + str(yc) + " " + str(w) + " " + str(h) + "\n"
#              print(self.save_dir + img_path[img_name_indl:img_name_indr] + ".txt")
              f.write(coord)

    def __len__(self):
          return len(self.imgs)

    def get_img_name(self, idx):
          return self.imgs[idx]

if __name__ == "__main__":
    dataset_path = "/content/detection/train"
    save_directory = "/content/data/labels/train/"

    dataset = AppleDataset(root_dir=dataset_path, save_dir = save_directory, transforms=None)

    num_images_to_process = min(1000, len(dataset))
    for i in range(num_images_to_process):
      dataset.get_item(i)

In [5]:
!zip -r labels.zip /content/data/labels

  adding: content/data/labels/ (stored 0%)
  adding: content/data/labels/train/ (stored 0%)
  adding: content/data/labels/train/20150921_131234_image661.txt (deflated 66%)
  adding: content/data/labels/train/20150921_132038_image856.txt (deflated 74%)
  adding: content/data/labels/train/20150921_131833_image126.txt (deflated 61%)
  adding: content/data/labels/train/20150919_174730_image121.txt (deflated 69%)
  adding: content/data/labels/train/20150919_174730_image646.txt (deflated 78%)
  adding: content/data/labels/train/20150919_174151_image1.txt (deflated 80%)
  adding: content/data/labels/train/20150921_131625_image336.txt (deflated 79%)
  adding: content/data/labels/train/20150921_131833_image266.txt (deflated 77%)
  adding: content/data/labels/train/20150921_131453_image221.txt (deflated 78%)
  adding: content/data/labels/train/20150921_131346_image986.txt (deflated 66%)
  adding: content/data/labels/train/20150921_131453_image81.txt (deflated 76%)
  adding: content/data/labels/t

In [ ]:
!cp /content/detection/train/images/* /content/data/images/train

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(data='/content/config.yaml', epochs=3)

100%|██████████| 6.23M/6.23M [00:00<00:00, 82.6MB/s]


Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/config.yaml, epochs=3, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None,

train: Scanning /content/data/labels/train.cache... 670 images, 0 backgrounds, 0 corrupt: 100%|██████████| 670/670 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/data/labels/train.cache... 670 images, 0 backgrounds, 0 corrupt: 100%|██████████| 670/670 [00:00<?, ?it/s]

...

        3/3         0G      2.013      1.108     0.9448       1019        640: 100%|██████████| 42/42 [12:42<00:00, 18.15s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [02:55<00:00,  8.36s/it]

                   all        670      28182      0.726      0.636      0.703      0.314



3 epochs completed in 0.806 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.0.231 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [02:26<00:00,  6.99s/it]


                   all        670      28182      0.725      0.636      0.703      0.314
Speed: 1.5ms preprocess, 162.8ms inference, 0.0ms loss, 9.9ms postprocess per image
Results saved to runs/detect/train3


In [ ]:
results = model("/content/data/images/train/20150919_174730_image376.png")


image 1/1 /content/data/images/train/20150919_174730_image376.png: 640x384 27 apples, 195.9ms
Speed: 3.4ms preprocess, 195.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)
